# Graphical Representations of Minconflicts Solutions

In [55]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path

plt.rcParams.update({
    "figure.dpi": 100,            # overridden per-figure on save
    "font.size": 12,
    "axes.grid": False,
})


In [56]:
ROOT = Path("..")  # adjust based on where the notebook lives
RESULTS_DIR = ROOT / "benchmarks" / "results"
FIG_DIR = ROOT / "figures"
FIG_DIR.mkdir(parents=True, exist_ok=True)

# For export sizing
EXPORT_DPI = 300
FIG_SIZE_SMALL = (6.67, 6.67)   # 6.67" * 300 dpi ≈ 2000 x 2000 px
FIG_SIZE_WIDE  = (8.0, 4.0)    # 2400 x 1200 px

print("Root Dir:", ROOT.resolve())

Root Dir: C:\Users\anton\Documents\VSCODE PROJECTS\CP468\CP468_TermProject


In [57]:
def plot_small_board_solution(
    sol_df: pd.DataFrame,
    out_path = None,
    *,
    show_attack_rays: bool = False,
    max_rays_queens = None,  # e.g. limit to first 5 to reduce clutter
) -> None:
    """
    Visualize an n-queens solution on a chess-like board.

    - Draws a checkerboard background.
    - Places a ♛ glyph at each queen position.
    - Optionally draws attack rays in all 8 directions from each queen
      until the edge of the board.

    Parameters
    ----------
    sol_df : DataFrame
        Must contain columns: 'n', 'c', 'row_final', 'd1', 'd2'.
    out_path : str or Path or None
        Where to save the figure (PNG). If None, the figure is not saved.
    show_attack_rays : bool, default False
        If True, draw 8-direction rays from each queen to board edges.
    max_rays_queens : int or None
        If not None, only draw rays for the first `max_rays_queens` queens
        in sol_df (to avoid clutter).
    """
    n = int(sol_df["n"].iloc[0])

    c = sol_df["c"].values
    r = sol_df["row_final"].values
    d1 = sol_df["d1"].values
    d2 = sol_df["d2"].values

    fig, axes = plt.subplots(
        1, 2, figsize=(10, 5), dpi=EXPORT_DPI,
        gridspec_kw={"width_ratios": [2, 1]}
    )
    ax_board, ax_table = axes

    # --------------------------
    # 1. Checkerboard background
    # --------------------------
    # board[row, col] = 0 or 1 pattern
    board = (np.add.outer(np.arange(n) % 2, np.arange(n) % 2)) % 2

    ax_board.imshow(
        board,
        cmap="Greys",  # dark/light squares
        origin="lower",
        extent=(-0.5, n - 0.5, -0.5, n - 0.5),
        alpha=0.3,
    )

    ax_board.set_title(f"n={n} solution: board view")
    ax_board.set_xlim(-0.5, n - 0.5)
    ax_board.set_ylim(-0.5, n - 0.5)
    ax_board.set_aspect("equal", adjustable="box")
    ax_board.set_xlabel("Column c")
    ax_board.set_ylabel("Row")

    # Draw grid lines around squares (minor ticks every 1)
    ax_board.set_xticks(np.arange(-0.5, n, 1), minor=True)
    ax_board.set_yticks(np.arange(-0.5, n, 1), minor=True)
    ax_board.grid(which="minor", color="black", linestyle="-", linewidth=0.2, alpha=0.4)
    ax_board.tick_params(which="major", length=0)
    ax_board.set_xticks(range(0, n, 5))
    ax_board.set_yticks(range(0, n, 5))

    # --------------------------------
    # 2. Draw queens as ♛ glyphs
    # --------------------------------
    for col, row in zip(c, r):
        ax_board.text(
            col,
            row,
            "♛",  # fallback: "Q" if font doesn't have this glyph
            ha="center",
            va="center",
            fontsize=20,           # tweak for taste / resolution
            color="black",
        )

    # --------------------------------
    # 3. Optional: attack rays in 8 directions
    # --------------------------------
    if show_attack_rays:
        # 8 directions: (dc, dr)
        directions = [
            ( 1,  0),  # right
            (-1,  0),  # left
            ( 0,  1),  # up
            ( 0, -1),  # down
            ( 1,  1),  # up-right
            ( 1, -1),  # down-right
            (-1,  1),  # up-left
            (-1, -1),  # down-left
        ]

        # Which queens get rays?
        if max_rays_queens is None:
            ray_indices = list(range(len(c)))
        else:
            ray_indices = list(range(min(max_rays_queens, len(c))))

        # Build a color palette for these queens
        # (tab20 gives you up to 20 nice distinct colors; beyond that we wrap)
        cmap = plt.get_cmap("tab20")
        num_colors = max(1, len(ray_indices))
        queen_colors = [
            cmap(i / max(1, num_colors - 1))
            for i in range(num_colors)
        ]

        for local_idx, queen_idx in enumerate(ray_indices):
            col0 = int(c[queen_idx])
            row0 = int(r[queen_idx])
            q_color = queen_colors[local_idx % num_colors]

            for dc, dr in directions:
                # Compute how far we can go before hitting the edge
                if dc > 0:
                    t_c = (n - 1) - col0
                elif dc < 0:
                    t_c = col0
                else:
                    t_c = np.inf

                if dr > 0:
                    t_r = (n - 1) - row0
                elif dr < 0:
                    t_r = row0
                else:
                    t_r = np.inf

                t_max = min(t_c, t_r)
                if not np.isfinite(t_max) or t_max <= 0:
                    continue

                end_c = col0 + dc * t_max
                end_r = row0 + dr * t_max

                ax_board.plot(
                    [col0, end_c],
                    [row0, end_r],
                    linewidth=3,
                    alpha=0.75,
                    color=q_color,
                )


    # --------------------------------
    # 4. Table of sample rows / diagonals
    # --------------------------------
    preview = sol_df[["c", "row_final", "d1", "d2"]].head(10)
    ax_table.axis("off")
    ax_table.set_title("Sample rows / diagonals")
    tbl = ax_table.table(
        cellText=preview.values,
        colLabels=preview.columns,
        loc="center",
    )
    tbl.auto_set_font_size(False)
    tbl.set_fontsize(8)

    if out_path is not None:
        fig.tight_layout()
        fig.savefig(out_path, bbox_inches="tight")
    plt.close(fig)

In [72]:
grid_df = pd.read_csv(RESULTS_DIR / "minconflicts_grid_improved.csv")

sol10_df = pd.read_csv(RESULTS_DIR / "solution_n10.csv")
sol20_df = pd.read_csv(RESULTS_DIR / "solution_n20.csv")
sol50_df = pd.read_csv(RESULTS_DIR / "solution_n50.csv")
sol1m_df = pd.read_csv(RESULTS_DIR / "solution_n1e6.csv")
display(grid_df)

,config_id,n,candidate_selector,candidate_count,structured_init,seed,k_sample,nbhd_width,max_restart,max_step_ratio,...,stats_solved,stats_steps,stats_restarts,stats_k_sample,stats_nbhd_width,stats_seed,stats_max_steps,timeout_hit,exception_occurred,exception_message
0,0,100,k_sample,32,0,42,32,0,3,500.0,...,1,485,0,32,0,42,50000,0,0,NaN
1,1,100,k_sample,32,0,123,32,0,3,500.0,...,1,346,0,32,0,123,50000,0,0,NaN
2,2,100,k_sample,32,0,67,32,0,3,500.0,...,1,254,0,32,0,67,50000,0,0,NaN
3,3,100,k_sample,32,1,42,32,0,3,500.0,...,1,281,0,32,0,42,50000,0,0,NaN
4,4,100,k_sample,32,1,123,32,0,3,500.0,...,1,394,0,32,0,123,50000,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,233,5000000,k_sample,2048,1,67,2048,0,3,8.0,...,1,5766822,0,2048,0,67,40000000,1,0,NaN
234,234,10000000,k_sample,1024,0,123,1024,0,3,20.0,...,1,7940105,0,1024,0,123,200000000,1,0,NaN
235,235,10000000,k_sample,1024,1,123,1024,0,3,20.0,...,1,12918914,0,1024,0,123,200000000,1,0,NaN
236,236,10000000,k_sample,10240,0,123,10240,0,3,20.0,...,1,6272285,0,10240,0,123,200000000,1,0,NaN


In [73]:

plot_small_board_solution(sol10_df, FIG_DIR / "n10_board.png", show_attack_rays=True)


In [74]:
def stratified_downsample(df: pd.DataFrame, every: int) -> pd.DataFrame:
    """
    Take every `every`-th column to reduce 1e6 rows -> ~1e6/every rows.
    """
    return df[df["c"] % every == 0].copy()


In [ ]:
def plot_large_solution_c_vs_row(sol_df: pd.DataFrame,
                                 color_by: str = "d1",
                                 stride: int = 100,
                                 out_path: Path | None = None) -> None:
    """
    Downsample columns by `stride` and plot c vs row_final with color mapping.
    color_by can be "d1" or "move_distance".
    """
    df = stratified_downsample(sol_df, every=stride)
    n = int(df["n"].iloc[0])

    c = df["c"].values
    r = df["row_final"].values
    color_vals = df[color_by].values

    # Normalize colors to [0,1]
    cmin, cmax = float(color_vals.min()), float(color_vals.max())
    denom = (cmax - cmin) if cmax > cmin else 1.0
    color_norm = (color_vals - cmin) / denom

    fig, ax = plt.subplots(figsize=FIG_SIZE_SMALL, dpi=EXPORT_DPI)
    sc = ax.scatter(
        c, r,
        c=color_norm,
        s=1,  # 1x1-ish at 2000x2000
        marker="s",
    )
    ax.set_title(f"n={n}: c vs row_final (color={color_by})")
    ax.set_xlabel("Column c")
    ax.set_ylabel("Row")
    # Optionally limit axes to [0, n-1]
    ax.set_xlim(0, n)
    ax.set_ylim(0, n)
    ax.invert_yaxis()

    cbar = fig.colorbar(sc, ax=ax)
    cbar.set_label(color_by)

    if out_path is not None:
        fig.tight_layout()
        fig.savefig(out_path, bbox_inches="tight")
    plt.close(fig)


In [87]:
plot_large_solution_c_vs_row(
    sol1m_df, color_by="d1", stride=100,
    out_path=FIG_DIR / "n1e6_c_vs_row_color_d1.png",
)

plot_large_solution_c_vs_row(
    sol1m_df, color_by="move_distance", stride=1,
    out_path=FIG_DIR / "n1e6_c_vs_row_color_movedist.png",
)


In [63]:
def plot_large_solution_diagonals(sol_df: pd.DataFrame,
                                  stride: int = 100,
                                  out_path: Path | None = None) -> None:
    df = stratified_downsample(sol_df, every=stride)
    n = int(df["n"].iloc[0])

    c = df["c"].values
    d1 = df["d1"].values
    d2 = df["d2"].values

    fig, ax = plt.subplots(figsize=FIG_SIZE_SMALL, dpi=EXPORT_DPI)
    ax.scatter(c, d1, s=1, marker=".", label="d1 = r - c")
    ax.scatter(c, d2, s=1, marker=".", label="d2 = r + c", alpha=0.5)

    ax.set_title(f"n={n}: c vs diagonal indices (downsampled)")
    ax.set_xlabel("Column c")
    ax.set_ylabel("Diagonal index")
    ax.legend(loc="upper right", markerscale=5)

    if out_path is not None:
        fig.tight_layout()
        fig.savefig(out_path, bbox_inches="tight")
    plt.close(fig)


In [64]:
def plot_large_solution_move_distance(sol_df: pd.DataFrame,
                                      stride: int = 100,
                                      out_path: Path | None = None) -> None:
    df = stratified_downsample(sol_df, every=stride)
    n = int(df["n"].iloc[0])
    c = df["c"].values
    md = df["move_distance"].values

    fig, ax = plt.subplots(figsize=FIG_SIZE_SMALL, dpi=EXPORT_DPI)
    ax.scatter(c, md, s=1, marker=".")
    ax.set_title(f"n={n}: c vs move_distance (downsampled)")
    ax.set_xlabel("Column c")
    ax.set_ylabel("|row_final - row_initial|")

    if out_path is not None:
        fig.tight_layout()
        fig.savefig(out_path, bbox_inches="tight")
    plt.close(fig)


In [65]:
def summarize_by_n_for_config(df: pd.DataFrame,
                              selector: str,
                              candidate_count: int,
                              structured_init: int) -> pd.DataFrame:
    sub = df[
        (df["candidate_selector"] == selector) &
        (df["candidate_count"] == candidate_count) &
        (df["structured_init"] == structured_init) &
        (df["solved"] == 1)
    ].copy()

    summary = (
        sub.groupby("n")
           .agg(
               steps_mean=("steps", "mean"),
               steps_std=("steps", "std"),
               time_mean=("total_time_sec", "mean"),
               time_std=("total_time_sec", "std"),
               solved_count=("solved", "sum"),
               runs=("solved", "count"),
           )
           .reset_index()
    )
    return summary


In [93]:
def plot_scaling_for_config(summary_df: pd.DataFrame,
                            label: str,
                            out_path: Path | None = None) -> None:
    fig, ax = plt.subplots(figsize=FIG_SIZE_WIDE, dpi=EXPORT_DPI)

    ns = summary_df["n"].values
    steps_mean = summary_df["steps_mean"].values
    steps_std = summary_df["steps_std"].values
    time_mean = summary_df["time_mean"].values
    time_std = summary_df["time_std"].values

    # --- helpers for nicer labels (no sci notation) ---
    def fmt_steps(y: float) -> str:
        # integer with thousands separators
        return f"{int(round(y)):,}"

    def fmt_time(y: float) -> str:
        # one decimal, no sci notation
        if y < 1:
            return f"{y:.2f}"
        else:
            return f"{y:.1f}"

    # Steps vs n (log-log)
    ax.errorbar(
        ns, steps_mean, yerr=steps_std,
        fmt="o-", label="steps (mean ± std)"
    )
    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.set_xlabel("n (log scale)")
    ax.set_ylabel("steps")

    # Data labels for steps
    for x, y in zip(ns, steps_mean):
        ax.annotate(
            fmt_steps(y),
            xy=(x, y),
            xytext=(10, 10),           # bigger offset
            textcoords="offset points",
            fontsize=8,
        )

    # Runtime on twin y-axis
    ax2 = ax.twinx()
    ax2.errorbar(
        ns, time_mean, yerr=time_std,
        fmt="s--", label="time (s, mean ± std)", color="tab:orange"
    )
    ax2.set_yscale("log")
    ax2.set_ylabel("runtime (seconds)")

    # Data labels for runtime
    for x, y in zip(ns, time_mean):
        ax2.annotate(
            fmt_time(y),
            xy=(x, y),
            xytext=(10, -18),          # bigger, separate offset
            textcoords="offset points",
            fontsize=8,
            color="tab:orange",
        )

    # Combine legends
    lines, labels_1 = ax.get_legend_handles_labels()
    lines2, labels_2 = ax2.get_legend_handles_labels()
    ax2.legend(lines + lines2, labels_1 + labels_2, loc="upper left")

    fig.suptitle(label)

    if out_path is not None:
        fig.tight_layout()
        fig.savefig(out_path, bbox_inches="tight")
    plt.close(fig)


In [94]:
opt_summary = summarize_by_n_for_config(
    grid_df,
    selector="k_sample",
    candidate_count=2048,
    structured_init=0,
)
plot_scaling_for_config(opt_summary,
                        label="Scaling for k_sample=2048, structured_init=0",
                        out_path=FIG_DIR / "scaling_optimal_config.png")


In [68]:
def summarize_at_n(df: pd.DataFrame, n_fixed: int) -> pd.DataFrame:
    return (
        df[(df["n"] == n_fixed) & (df["solved"] == 1)]
        .groupby(["candidate_selector", "candidate_count", "structured_init"])
        .agg(
            steps_mean=("steps", "mean"),
            steps_std=("steps", "std"),
            time_mean=("total_time_sec", "mean"),
            time_std=("total_time_sec", "std"),
        )
        .reset_index()
    )


In [69]:
def plot_steps_vs_candidates_selector_effect(summary_df: pd.DataFrame,
                                             n_fixed: int,
                                             structured_init: int,
                                             out_path: Path | None = None) -> None:
    fig, ax = plt.subplots(figsize=FIG_SIZE_WIDE, dpi=EXPORT_DPI)

    for selector in ["k_sample", "nbhd"]:
        sub = summary_df[
            (summary_df["candidate_selector"] == selector) &
            (summary_df["structured_init"] == structured_init)
        ]
        if sub.empty:
            continue
        x = sub["candidate_count"].values
        y = sub["steps_mean"].values
        yerr = sub["steps_std"].values
        ax.errorbar(x, y, yerr=yerr, marker="o", linestyle="-", label=selector)

    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.set_xlabel("candidate_count")
    ax.set_ylabel("steps (mean ± std)")
    ax.set_title(f"n={n_fixed}: candidate_count vs steps (structured_init={structured_init})")
    ax.legend()

    if out_path is not None:
        fig.tight_layout()
        fig.savefig(out_path, bbox_inches="tight")
    plt.close(fig)


In [70]:
def plot_steps_vs_candidates_init_effect(summary_df: pd.DataFrame,
                                         n_fixed: int,
                                         selector: str,
                                         out_path: Path | None = None) -> None:
    fig, ax = plt.subplots(figsize=FIG_SIZE_WIDE, dpi=EXPORT_DPI)

    for struct in [0, 1]:
        sub = summary_df[
            (summary_df["candidate_selector"] == selector) &
            (summary_df["structured_init"] == struct)
        ]
        if sub.empty:
            continue
        x = sub["candidate_count"].values
        y = sub["steps_mean"].values
        yerr = sub["steps_std"].values
        label = f"{selector}, structured_init={struct}"
        ax.errorbar(x, y, yerr=yerr, marker="o", linestyle="-", label=label)

    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.set_xlabel("candidate_count")
    ax.set_ylabel("steps (mean ± std)")
    ax.set_title(f"n={n_fixed}: candidate_count vs steps (selector={selector})")
    ax.legend()

    if out_path is not None:
        fig.tight_layout()
        fig.savefig(out_path, bbox_inches="tight")
    plt.close(fig)


In [71]:
n_fixed = 1_000_000
summary_1m = summarize_at_n(grid_df, n_fixed=n_fixed)

plot_steps_vs_candidates_selector_effect(
    summary_1m, n_fixed, structured_init=0,
    out_path=FIG_DIR / "n1e6_candidates_vs_steps_selector.png",
)

plot_steps_vs_candidates_init_effect(
    summary_1m, n_fixed, selector="k_sample",
    out_path=FIG_DIR / "n1e6_candidates_vs_steps_init.png",
)
